In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import math

import seaborn as sns       
from numpy import nan
from scipy import stats
from scipy.stats import  norm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline  

from sklearn import preprocessing
from sklearn import linear_model, svm, gaussian_process
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


data_train = pd.read_csv("/Users/mac/Documents/GitHub/Dive-into-DL-PyTorch/data/kaggle_house/train.csv")

 # 删除缺失数据

In [2]:
for i in data_train.columns[1:]:
    if data_train[i].isnull().sum()>0:
        print(i,data_train[i].isnull().sum())

LotFrontage 259
Alley 1369
MasVnrType 8
MasVnrArea 8
BsmtQual 37
BsmtCond 37
BsmtExposure 38
BsmtFinType1 37
BsmtFinType2 38
Electrical 1
FireplaceQu 690
GarageType 81
GarageYrBlt 81
GarageFinish 81
GarageQual 81
GarageCond 81
PoolQC 1453
Fence 1179
MiscFeature 1406


In [3]:

Deldata= data_train[[column for column in data_train if data_train[column].count()/len(data_train)>=0.3]]

del Deldata['Id']
print("删除的属性", end=" ")
for c in data_train.columns:
    if c not in Deldata.columns:
        print(c, end=", ")
print('\n')
data_train = Deldata

删除的属性 Id, Alley, PoolQC, Fence, MiscFeature, 



In [4]:
data_train.shape

(1460, 76)

In [5]:
numCol='LotFrontage/LotArea/MasVnrArea/BsmtFinSF1/BsmtFinSF2/BsmtUnfSF/TotalBsmtSF/1stFlrSF/2ndFlrSF/LowQualFinSF/GrLivArea/BsmtFullBath/BsmtHalfBath/FullBath/HalfBath/BedroomAbvGr/KitchenAbvGr/TotRmsAbvGrd/Fireplaces/GarageCars/GarageArea/WoodDeckSF/OpenPorchSF/EnclosedPorch/3SsnPorch/ScreenPorch/PoolArea/MiscVal/MoSold/YrSold/YearBuilt/YearRemodAdd/GarageYrBlt/SalePrice'
num=numCol.split('/')
numFeature=data_train[num]
numFeature.shape

(1460, 34)

In [6]:

Nominal='MSSubClass/MSZoning/Street/LotShape/LandContour/Utilities/LotConfig/LandSlope/Neighborhood/Condition1/Condition2/BldgType/HouseStyle/OverallQual/OverallCond/RoofStyle/RoofMatl/Exterior1st/Exterior2nd/MasVnrType/ExterQual/ExterCond/Foundation/BsmtQual/BsmtCond/BsmtExposure/BsmtFinType1/BsmtFinType2/Heating/HeatingQC/CentralAir/Electrical/KitchenQual/Functional/FireplaceQu/GarageType/GarageFinish/GarageQual/GarageCond/PavedDrive/SaleType/SaleCondition'
NomFeature=data_train[Nominal.split('/')]
NomFeature.shape

(1460, 42)

# 转换数据类型

In [7]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
for i in NomFeature.columns:
    if data_train[i].dtype=='O':
        data_train[i]=number.fit_transform(data_train[i].astype('str'))
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null int64
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null int64
LotShape         1460 non-null int64
LandContour      1460 non-null int64
Utilities        1460 non-null int64
LotConfig        1460 non-null int64
LandSlope        1460 non-null int64
Neighborhood     1460 non-null int64
Condition1       1460 non-null int64
Condition2       1460 non-null int64
BldgType         1460 non-null int64
HouseStyle       1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null int64
RoofMatl         1460 non-null int64
Exterior1st      1460 non-null int64
Exterior2nd      1460 non-null int64
MasVnrType       1460 non-null int64
Mas

# 利用回归模型预测结果

### 对选中的和销售价格相关性高的特征数据操作，分割训练集和测试集

In [8]:
#cols = ['OverallQual','GrLivArea', 'GarageCars','TotalBsmtSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']

cols=['GrLivArea','GarageCars','TotalBsmtSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd','OverallQual','Foundation','CentralAir',
'Electrical','PavedDrive','RoofStyle','SaleCondition','Neighborhood','HouseStyle']
x = data_train[cols].values
y = data_train['SalePrice'].values
x_scaled = preprocessing.StandardScaler().fit_transform(x)
y_scaled = preprocessing.StandardScaler().fit_transform(y.reshape(-1,1))
X_train,X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.33, random_state=42)

In [9]:
clfs = {
        'svm':svm.SVR(), 
        'RandomForestRegressor':RandomForestRegressor(n_estimators=400),
        'BayesianRidge':linear_model.BayesianRidge()
       }
for clf in clfs:
    try:
        clfs[clf].fit(X_train, y_train)
        y_pred = clfs[clf].predict(X_test)
        print(clf + " cost:" + str(np.sum(y_pred-y_test)/len(y_pred)))
        #print(clf + " cost:" + str(np.sum(y_pred-y_test)/len(y_pred)) )
    except Exception as e:
        print(clf + " Error:")
        print(str(e))

svm cost:-12.920976554177884
RandomForestRegressor cost:1.2494114454346976
BayesianRidge cost:-14.263246442318541


# 选择模型进行预测，并保存模型

In [10]:
cols=['GrLivArea','GarageCars','TotalBsmtSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd','OverallQual','Foundation','CentralAir',
'Electrical','PavedDrive','RoofStyle','SaleCondition','Neighborhood','HouseStyle']
x = data_train[cols].values
y = data_train['SalePrice'].values
X_train,X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

clf =linear_model.BayesianRidge()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


# MAE和RMSE评估预测结果

In [11]:
mae=sum(abs(y_pred - y_test))/len(y_pred)
rmse=np.sqrt(sum(pow(y_pred - y_test,2)/len(y_pred)))
print(math.log(mae))
print(math.log(rmse))

10.123019482600032
10.579831695443538


# 获取之前训练的模型

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model, svm, gaussian_process

# 之前训练的模型
rfr = clf

In [13]:
data_test = pd.read_csv("/Users/mac/Documents/GitHub/Dive-into-DL-PyTorch/data/kaggle_house/test.csv")

In [14]:
data_test[cols].isnull().sum()

GrLivArea        0
GarageCars       1
TotalBsmtSF      1
FullBath         0
TotRmsAbvGrd     0
YearBuilt        0
YearRemodAdd     0
OverallQual      0
Foundation       0
CentralAir       0
Electrical       0
PavedDrive       0
RoofStyle        0
SaleCondition    0
Neighborhood     0
HouseStyle       0
dtype: int64

# 对缺失数据进行平均值填充

In [15]:
for column in list(data_test[cols].columns[data_test[cols].isnull().sum() > 0]):
    meanval = data_test[column].mean()
    data_test[column].fillna(meanval, inplace=True)
data_test_x=data_test[cols]
data_test_x.isnull().sum()

GrLivArea        0
GarageCars       0
TotalBsmtSF      0
FullBath         0
TotRmsAbvGrd     0
YearBuilt        0
YearRemodAdd     0
OverallQual      0
Foundation       0
CentralAir       0
Electrical       0
PavedDrive       0
RoofStyle        0
SaleCondition    0
Neighborhood     0
HouseStyle       0
dtype: int64

In [16]:
Nominal='GrLivArea/GarageCars/TotalBsmtSF/FullBath/TotRmsAbvGrd/YearBuilt/YearRemodAdd/OverallQual/Foundation/CentralAir/Electrical/PavedDrive/RoofStyle/SaleCondition/Neighborhood/HouseStyle'
NomFeature=data_test_x[Nominal.split('/')]
NomFeature.shape

(1459, 16)

# 把字符串类型数据转换为浮点型数据

In [17]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
for i in NomFeature.columns:
    if data_test_x[i].dtype=='O':
        data_test_x[i]=number.fit_transform(data_test_x[i].astype('str'))
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

# 使用保存的模型进行预测

In [18]:
x = data_test_x.values
y_te_pred = rfr.predict(x)
print(y_te_pred)

[106007.81723471 164180.05932099 173396.19970668 ... 143340.53346042
 113190.61191245 242944.21965464]


# 保存数据到csv文件

In [19]:
prediction = pd.DataFrame(y_te_pred, columns=['SalePrice'])
result = pd.concat([ data_test['Id'], prediction], axis=1)
# result = result.drop(resultlt.columns[0], 1)
result.columns

Index(['Id', 'SalePrice'], dtype='object')

In [20]:
result.to_csv('./Predictions.csv', index=False)